# Solution for Global Minimial

In [1]:
from time import time

$$ f(x)=x^3-60x^2-4x+6 $$, 求x在[0,100]范围内f(x)最小值

$$f'(x) = 3x^2 - 120x - 4$$

$$ x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)} $$

# Newton for f'

In [2]:
start = time()

In [3]:
x0 = 100
x1 = x0 - (3*(x0**2) - 120 * x0 - 4) / (6 * x0 - 120)
f_val = (x0 ** 3) - 60 * (x0**2) - 4 * x0 + 6

In [4]:
while f_val > 1e-8:
    x1 = x0 - (3*(x0**2) - 120 * x0 - 4) / (6 * x0 - 120)
    x0 = x1
    f_val = 3*(x0**2) - 120 * x0 - 4

In [5]:
print("Newton method for f' is about : %0.2f seconds" % (time() - start))

Newton method for f' is about : 0.02 seconds


In [6]:
x0

40.03330560175586

In [7]:
(x0 ** 3) - 60 * (x0**2) - 4 * x0 + 6

-32154.06662967586

# Gradient Descent

$$ f(x)=x^3-60x^2-4x+6 $$

In [8]:
import tensorflow as tf
from tensorflow.train import AdamOptimizer

In [9]:
start = time()

In [10]:
x = tf.get_variable('x', initializer=tf.constant(100.0))
y = x * x * x - 60 * x * x - 4 * x + 6

In [11]:
optimizer = AdamOptimizer(learning_rate=1e-2).minimize(y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(50000):
        sess.run(optimizer)
    print(sess.run(x))

40.034805


In [12]:
print("Gradient Descent for y is about : %0.2f seconds" % (time() - start))

Gradient Descent for y is about : 11.93 seconds


# Binary Search

similiar to Newton method from [0, 100]. Somehow simple, just skip it

# MCTS

In [13]:
import random

In [14]:
start = time()

In [15]:
min_val = (lambda x: x**3 + 6)(100)
solution = 100
best_step = 0
delta_value = 100000; delta_no_improve = 0
fv_pre = (lambda v: v**3 - 60 * (v**2) - 4 * v + 6)(solution)
i = 0
# for i in range(1000000):
#     1. only integer number
#     v = random.randrange(0, 100.0)
#     2. sample uniform between [0, 1) * 100.0
while delta_no_improve < 200000:
    v = random.random() * 100
    fv = (lambda v: v**3 - 60 * (v**2) - 4 * v + 6)(v)
    if min_val > fv:
        min_val = fv
        solution = v
        best_step = i
        delta_value = 100000; delta_no_improve = 0
    else:
        if abs(fv_pre - fv) < delta_value:
            delta_value = abs(fv_pre - fv)
        else:
            delta_no_improve += 1
    fv_pre = fv; i += 1

In [16]:
print("Random MCTS is about : %0.2f seconds" % (time() - start))

Random MCTS is about : 0.36 seconds


In [17]:
delta_value

0.12307333269200171

In [18]:
delta_no_improve

200000

In [19]:
min_val

-32154.066629492823

In [20]:
best_step

151372

In [21]:
solution

40.03325041638389

# MCTS - Exploration with f' as penality

Sample with **Uniform Distribution** to select better seed

In [22]:
import random
import numpy as np

In [23]:
start = time()

In [24]:
min_val = (lambda x: x**3 + 6)(100)
solution = 100
best_penality = 1000000
best_step = 0
no_improve = 0
seed = 0
minv_region = 1; maxv_region = -1; sample_region_n = 0
delta_value = 100000; delta_no_improve = 0
fv_pre = (lambda v: v**3 - 60 * (v**2) - 4 * v + 6)(solution)
i = 0

while delta_no_improve < 200000:
# for i in range(1000000):
    if no_improve < 10000 or sample_region_n < 100:
        seed = random.random()
    else:
        # uniform seems still not very good, how about gaussion?
        seed = np.random.uniform(minv_region, maxv_region)
    v = seed * 100
    fv = (lambda v: v**3 - 60*(v**2) - 4 * v + 6)(v)
    penality = (lambda x: x**3 - 60*(x**2) - 4*x + 6)(v)
    if abs(penality) < best_penality:
        best_penality = abs(penality)
        no_improve = 0
        minv_region = 1; maxv_region = -1; sample_region_n = 0
    else:
        no_improve += 1
        if no_improve > 10000:
            if v > maxv_region:
                maxv_region = seed
            if v < minv_region:
                minv_region = seed
            sample_region_n += 1
        
    if min_val > fv:
        min_val = fv
        solution = v
        best_step = i
    else:
        if abs(fv_pre - fv) < delta_value:
            delta_value = abs(fv_pre - fv)
        else:
            delta_no_improve += 1
    fv_pre = fv; i += 1

In [25]:
print("MCTS with uniform sampling for exploration is about : %0.2f seconds" % (time() - start))

MCTS with uniform sampling for exploration is about : 0.61 seconds


In [26]:
delta_value

0.0

In [27]:
delta_no_improve

200000

In [28]:
min_val

-32154.066376910876

In [29]:
solution

40.03535635828786

In [30]:
best_step

10633

# MCTS - Exploration with f' as penality

Sample with **Gaussion Distribution** to select better seed

Too costy for calcuation of $\delta$ for Gaussian distribution, set $\delta$ = 0.01

In [31]:
import random
import numpy as np

In [32]:
start = time()

In [33]:
min_val = (lambda x: x**3 + 6)(100)
solution = 100
best_penality = 1000000
best_step = 0
no_improve = 0
seed = 0
minv_region = 1; maxv_region = -1; sample_region_n = 0; sample_mean = 0
delta_value = 100000; delta_no_improve = 0
fv_pre = (lambda v: v**3 - 60 * (v**2) - 4 * v + 6)(solution)
i = 0

while delta_no_improve < 200000:
# for i in range(1000000):
    if no_improve < 10000 or sample_region_n < 1000:
        seed = random.random()
    else:
        # uniform seems still not very good, how about gaussion?
        seed = np.random.normal(sample_mean, 0.01, 1)[0]
    v = seed * 100
    fv = (lambda v: v**3 - 60*(v**2) - 4 * v + 6)(v)
    penality = (lambda x: x**3 - 60*(x**2) - 4*x + 6)(v)
    if abs(penality) < best_penality:
        best_penality = abs(penality)
        no_improve = 0
        minv_region = 1; maxv_region = -1; sample_region_n = 0
    else:
        no_improve += 1
        if no_improve > 10000:
            if v > maxv_region:
                maxv_region = seed
            if v < minv_region:
                minv_region = seed
            sample_mean = (sample_region_n * sample_mean + seed) / (sample_region_n + 1)
            sample_region_n += 1
        
    if min_val > fv:
        min_val = fv
        solution = v
        best_step = i
    else:
        if abs(fv_pre - fv) < delta_value:
            delta_value = abs(fv_pre - fv)
        else:
            delta_no_improve += 1
    fv_pre = fv; i += 1

In [34]:
print("MCTS with gaussion sampling for exploration is about : %0.2f seconds" % (time() - start))

MCTS with gaussion sampling for exploration is about : 1.74 seconds


In [35]:
delta_value

0.04919455462368205

In [36]:
delta_no_improve

200000

In [37]:
min_val

-32154.06634607454

In [38]:
solution

40.03547784935626

In [39]:
best_step

9028